In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

#This line will hide code by default when the notebook is exported as HTML
#di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import hail as hl
hl.init(tmp_dir='/net/scratch/people/plggosborcz')

from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()


import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis

output_notebook() 

### note - paths in this notebook are outdated

In [ ]:
files = !ls /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-vcfs/*gz

In [ ]:
names = []

for i in files:
    i = i.split('/')
    i = i[9]
    i = i.split('.')
    i = i[0]
    names.append(i)

In [ ]:
#importing part - 10

small_mts = []

small = !ls /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-vcfs/part10/vcf-part-10-17-to-20/*gz

for f in small:
    mt = hl.import_vcf(f, reference_genome='GRCh38', filter='chr9[ \t]138334443', force_bgz = True, array_elements_required = False)
    print(mt.count())
    small_mts.append(mt)
    
mt_small = hl.MatrixTable.union_rows(*small_mts)

In [ ]:
mt_small.count()

In [ ]:
larger_mts = []

larger = !ls /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-vcfs/part10/*gz

for f in larger:
    mt = hl.import_vcf(f, reference_genome='GRCh38', filter='chr9[ \t]138334443', force_bgz = True, array_elements_required = False)
    print(mt.count())
    larger_mts.append(mt)
    
mt_larger = hl.MatrixTable.union_rows(*larger_mts)

In [ ]:
mt_larger.count()

In [ ]:
mt_larger = mt_larger.filter_cols(mt_larger.s != '464')

In [ ]:
mt_10 = mt_larger.union_rows(mt_small)

In [ ]:
mt_10.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-mts/part010.mt')

In [ ]:
for idx, f in enumerate(files):
    hl.import_vcf(f, reference_genome='GRCh38', filter='chr9[ \t]138334443', force_bgz = True, array_elements_required = False).write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-mts/'+names[idx]+'.mt')

In [ ]:
mts = []
for i in names:
    i = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-mts/'+i+'.mt')
    mts.append(i)

In [ ]:
for mt in mts:
    pprint(mt.aggregate_rows(hl.agg.counter(mt.locus.contig)))

In [ ]:
rpmk = hl.import_bed('repeatmasker_all', reference_genome='GRCh38', skip_invalid_intervals=True)

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-mts/part010.mt')

In [ ]:
mt = mt.filter_rows(hl.is_defined(rpmk[mt.locus]), keep = False)
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-rpmk/part010.mt')
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-rpmk/part010.mt')

In [ ]:
for idx, mt in enumerate(mts):
    print(mt.count())
    mt = mt.filter_rows(hl.is_defined(rpmk[mt.locus]), keep = False)
    mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-rpmk/'+names[idx]+'.mt')
    mts[idx] = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-rpmk/'+names[idx]+'.mt')

In [ ]:
for mt in mts:
    print(mt.count())

In [ ]:
table = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/vcf_preprocessing/gnomad.genomes.r3.0.coverage.summary.tsv.bgz', impute = True)

table = table.select(table.locus, table.over_1)

table = table.annotate(locus = hl.parse_locus(table.locus, reference_genome='GRCh38'))

table = table.filter(table.over_1 >0.9)

table = table.key_by(table.locus)

In [ ]:
mt = mt.filter_rows(hl.is_defined(table[mt.locus]), keep = True)
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/part010.mt')
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/part010.mt')

In [ ]:
for idx, mt in enumerate(mts):
    print(mt.count())
    mt = mt.filter_rows(hl.is_defined(table[mt.locus]), keep = True)
    mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/'+names[idx]+'.mt')
    mts[idx] = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/'+names[idx]+'.mt')

## Start from here 

In [ ]:
files = !ls /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/

In [ ]:
mts = []

for f in files:
    f = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/'+f)
    mts.append(f)

In [ ]:
gnomad = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/gnomad/gnomad_selected_filtered.ht')

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/part010.mt')
mt = mt.select_rows(mt.rsid, mt.info.AC, mt.info.AF, mt.info.AN)
mt = hl.split_multi_hts(mt)
mt = mt.annotate_rows(gnomad_v3 = gnomad[mt.row_key])
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-gnomad/part010.mt')

In [ ]:
for idx, mt in enumerate(mts):
    mt = mt.select_rows(mt.rsid, mt.info.AC, mt.info.AF, mt.info.AN)
    mt = hl.split_multi_hts(mt)
    mt = mt.annotate_rows(gnomad_v3 = gnomad[mt.row_key])
    mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-gnomad/'+files[idx])

Now - see example annotations and annotate with nearby genes and HPO

In [ ]:
mts = []

for f in files:
    f = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-gnomad/'+f)
    mts.append(f)

In [ ]:
genes = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/vcf_preprocessing/genecode_v32.ht')

In [ ]:
genes = genes.filter(hl.is_valid_contig(genes['hg38.knownGene.chrom'], reference_genome='GRCh38'))

In [ ]:
hpo = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/vcf_preprocessing/hpo.tsv', impute = True, no_header=True)

In [ ]:
start = genes['hg38.knownGene.txStart']
stop =  genes['hg38.knownGene.txEnd']

genes = genes.transmute(interval = 
                        hl.locus_interval(genes['hg38.knownGene.chrom'], 
                                          start,
                                          stop,
                                          reference_genome='GRCh38', includes_start=False))

genes = genes.key_by(genes['hg38.kgXref.geneSymbol'])

hpo = hpo.key_by(hpo.f0)

genes = genes.annotate(hpo = hpo.index(genes['hg38.kgXref.geneSymbol'], all_matches = True)['f1'])

genes = genes.key_by(genes.interval)

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-gnomad/part010.mt')
mt = mt.annotate_rows(within_gene = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hg38.kgXref.geneSymbol'])))
mt= mt.annotate_rows(hpo = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hpo'])))
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-hpo/part010.mt')

In [ ]:
for idx, mt in enumerate(mts):
    mts[idx] = mt.annotate_rows(within_gene = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hg38.kgXref.geneSymbol'])))
    mts[idx] = mt.annotate_rows(hpo = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hpo'])))
    mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-hpo/'+files[idx])

In [ ]:
files = !ls /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-hpo/

In [ ]:
mts = []

for f in files:
    f = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-hpo/'+f)
    f = f.filter_cols(f.s != '464')
    mts.append(f)

## one last step - join + perform PCA 

In [ ]:
mt_full = hl.MatrixTable.union_rows(*mts)

In [ ]:
mt_full.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS.mt')

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS.mt')

In [ ]:
mt = mt.annotate_rows(info = hl.struct(AC = mt.AC, AF = mt.AF, gnomad_nfe_AF = mt.gnomad_v3.v3_nfe.AF, gnomad_nfe_homozygote_count = mt.gnomad_v3.v3_nfe.homozygote_count, highest_consequence = mt.gnomad_v3.most_severe_consequence,
                                  exon = mt.gnomad_v3.transcript_consequences.exon,
                                  transcript_impact = mt.gnomad_v3.transcript_consequences.impact, lof = mt.gnomad_v3.transcript_consequences.lof,
                                  pholyphen_prediction = mt.gnomad_v3.transcript_consequences.polyphen_prediction, polyphen_score = mt.gnomad_v3.transcript_consequences.polyphen_score, 
                                  sift_prediction = mt.gnomad_v3.transcript_consequences.sift_prediction, sift_score = mt.gnomad_v3.transcript_consequences.sift_score,
                                  transcript_id = mt.gnomad_v3.transcript_consequences.transcript_id, variant_class = mt.gnomad_v3.variant_class, gene_symbol = mt.within_gene, hpo = hl.flatten(mt.hpo)))

In [ ]:
hl.export_vcf(mt, '/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS_all.vcf.bgz')

In [ ]:
samples = mt.s.collect()

In [ ]:
for s in samples:
    mt_s = mt.filter_cols(mt.s == s)
    hl.export_vcf(mt_s,'/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-annotated-vcf/GTS-individual-vcf'+s+'vcf.bgz')

In [ ]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt_sample.GT)
mt_sample = mt_sample.annotate_cols(scores = pcs[mt_sample.s].scores)

p = hl.plot.scatter(mt_sample.scores[0],
                    mt_sample.scores[1],
                    label=mt_sample.s,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

In [ ]:
mt_sample = mt_sample.annotate_cols(group = hl.cond(mt_sample.s.contains("S"), "old", "new"))

In [ ]:
p = hl.plot.scatter(mt_sample.scores[3],
                    mt_sample.scores[4],
                    label=mt_sample.group,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)